In [1]:
# # python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'panda'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result-btf-mvik10-mpto3"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

def run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=['None', 'Tool', 'ToolReach', 'Full']):
    N_DAT = I_END-I_START
    ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
    file_list = sorted(os.listdir(DATASET_PATH))

    gtimer = GlobalTimer.instance()
    gtimer.reset(scale=1, timeunit='s')

    gtimer.tic("full_loop")
    for fid in range(I_START, I_END):
        e_step_list = []
        for i_c, cname in enumerate(CNAME_LIST):
            gtimer.tic("step")
            p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                                  '--data_idx', str(fid), '--file_option', FILE_OPTION, '--cname', cname,]
                                 + common_args)
            p.wait()
            e_step = np.round(gtimer.toc("step"), 1)
            e_step_list.append(e_step)
            print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")

        print("")
        print("========== {} / {} (eta: {} / {} s) =========".format(
            fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

## Common params

In [2]:
RTYPE = "panda"
DAT_DIR = '20210917-113211'
I_START = 10
I_END = 30


CNAME_LIST = ['None', 'Tool', 'ToolReach', 'Full']
# CNAME_LIST = ['Full'] #,'Pairwise']
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 3
MAX_TIME = 100
MAX_ITER = 100

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO= 100
MAX_SKELETONS = 30
TIMED_COMPLETE = False
STACK_TIMELOG = True
IK_TRY_NUM = 10
VERBOSE = False
USE_MOVEIT_IK = True


common_args = [
            '--rtype', RTYPE,
            '--dat_root', DAT_ROOT,
            '--res_root', RES_ROOT,
            '--dat_dir', DAT_DIR,
            '--VISUALIZE', str(VISUALIZE),
            '--PLAY_RESULT', str(PLAY_RESULT),
            '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
            '--MAX_TIME', str(MAX_TIME),
            '--MAX_ITER', str(MAX_ITER), 
            '--GRASP_SAMPLE', str(GRASP_SAMPLE),
            '--STABLE_SAMPLE', str(STABLE_SAMPLE),
            '--SEARCH_SAMPLE_RATIO', str(SEARCH_SAMPLE_RATIO),
            '--MAX_SKELETONS', str(MAX_SKELETONS),
            '--TIMED_COMPLETE', str(TIMED_COMPLETE),
            '--STACK_TIMELOG', str(STACK_TIMELOG),
            '--IK_TRY_NUM', str(IK_TRY_NUM),
            '--VERBOSE', str(VERBOSE),
            '--USE_MOVEIT_IK', str(USE_MOVEIT_IK)]

# obj1 obs0

In [3]:
FILE_OPTION = "obj_1c_obs0"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_1c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

FILE_OPTION = "obj_3c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args, CNAME_LIST=CNAME_LIST)

------- 14.0    7.7     7.9     7.4     -------
========== 11 / 20 (eta: 37.1 / 741.1 s) =========
------- 8.1     6.7     11.2    7.8     -------
========== 12 / 20 (eta: 70.9 / 709.1 s) =========
------- 12.3    7.3     8.3     6.9     -------
========== 13 / 20 (eta: 105.8 / 705.0 s) =========
------- 7.7     7.4     12.2    11.9    -------
========== 14 / 20 (eta: 145.0 / 725.0 s) =========
------- 8.7     7.2     9.8     10.1    -------
========== 15 / 20 (eta: 180.9 / 723.4 s) =========
------- 6.5     7.1     6.8     7.5     -------
========== 16 / 20 (eta: 208.8 / 696.0 s) =========
------- 12.8    23.8    7.6     6.9     -------
========== 17 / 20 (eta: 260.0 / 742.7 s) =========
------- 6.7     6.3     7.3     6.8     -------
========== 18 / 20 (eta: 287.1 / 717.8 s) =========
------- 6.8     6.8     7.5     7.0     -------
========== 19 / 20 (eta: 315.2 / 700.4 s) =========
------- 15.9    6.3     10.2    15.1    -------
========== 20 / 20 (eta: 362.8 / 725.5 s) =========
--

------- 6.8     6.7     7.0     6.9     -------
========== 13 / 20 (eta: 138.8 / 925.2 s) =========
------- 8.1     10.4    7.9     8.1     -------
========== 14 / 20 (eta: 173.3 / 866.7 s) =========
------- 8.6     10.1    7.4     11.7    -------
========== 15 / 20 (eta: 211.2 / 844.9 s) =========
------- 31.5    52.4    12.9    10.3    -------
========== 16 / 20 (eta: 318.4 / 1061.3 s) =========
------- 13.1    7.7     10.5    7.8     -------
========== 17 / 20 (eta: 357.6 / 1021.6 s) =========
------- 106.5   106.1   106.4   39.1    -------
========== 18 / 20 (eta: 715.7 / 1789.4 s) =========
------- 6.7     6.7     8.6     7.3     -------
========== 19 / 20 (eta: 745.1 / 1655.7 s) =========
------- 7.7     11.1    7.3     10.5    -------
========== 20 / 20 (eta: 781.8 / 1563.5 s) =========
------- 93.6    106.0   106.4   106.5   -------
========== 21 / 20 (eta: 1194.2 / 2171.4 s) =========
------- 7.6     6.6     6.8     7.1     -------
========== 22 / 20 (eta: 1222.4 / 2037.4 s) =